First Jupyter Notebook For Debugging Purposes!

In [ ]:
using Plots

# Constants: (SI Units)
const M_S = 1.989e30 # The Mass of The Sun (Kg)

In [ ]:
const M_E = 5.972e24 # The Mass of The Earth (Kg)

In [ ]:
const M_M = 7.34767309e22 # The Mass of The Moon (Kg)

In [ ]:
const G = 6.6743e-11 # Gravitational Constant (m^3kg^-1s^-2)

In [ ]:
const AU = 149597870700 # Mean Distance Between The Sun And The Earth (m)

In [ ]:
const SEMD = AU # Sun-Earth Mean Distance = 149597870.700 Km

In [ ]:
const EMMD = AU / 391.1055443137255 # Earth-Moon Mean Distance = 382,500 Km

In [ ]:
const M_i = 5.15 # The Moon moves in an approximately elliptic orbit inclined 
# at about five degrees to the plane of the ecliptic. (In Degrees)

In [ ]:
## Linear Velocity of The Earth In Orbit Around The Sun:
# This equation can be proved with the help of polar
# coordinates and two assumptions:
# 1- r is constant in orbital motion (r = AU)
# 2- Theta_Dot is constant in orbital motuon (Theta_Dot = cte.)
const V_E0 = (G * M_S / SEMD)^0.5 # Earth Initial Velocity

In [ ]:
const V_M0 = (G * M_E / EMMD)^0.5 + V_E0 # Moon Initial Velocity

In [ ]:
# State-Space Matrix Calculation Function:
function f(x_var)
    D_A1_A2 = 1 / (((x_var[1] - x_var[4])^2 + (x_var[2] - x_var[5])^2 + (x_var[3] - x_var[6])^2)^(3 / 2))
    D_B1 = 1 / ((x_var[1]^2 + x_var[2]^2 + x_var[3]^2)^(3 / 2)) # Inverse of The Distance of The Moon
    D_B2 = 1 / ((x_var[4]^2 + x_var[5]^2 + x_var[6]^2)^(3 / 2)) # Inverse of The Distance of The Earth

    A1 = (G * M_E) / D_A1_A2
    A2 = (G * M_M) / D_A1_A2
    B1 = (G * M_S) / D_B1
    B2 = (G * M_S) / D_B2

    Z1 = -B1 - A1
    Z2 = -B2 - A2

    # Stat-Space Matrix
    A = [0 0 0 0 0 0 1 0 0 0 0 0
        0 0 0 0 0 0 0 1 0 0 0 0
        0 0 0 0 0 0 0 0 1 0 0 0
        0 0 0 0 0 0 0 0 0 1 0 0
        0 0 0 0 0 0 0 0 0 0 1 0
        0 0 0 0 0 0 0 0 0 0 0 1
        Z1 0 0 A1 0 0 0 0 0 0 0 0
        0 Z1 0 0 A1 0 0 0 0 0 0 0
        0 0 Z1 0 0 A1 0 0 0 0 0 0
        A2 0 0 Z2 0 0 0 0 0 0 0 0
        0 A2 0 0 Z2 0 0 0 0 0 0 0
        0 0 A2 0 0 Z2 0 0 0 0 0 0]

    f_x = A * x_var
    return f_x
end

In [ ]:
# Initialize System:
Day_Hours = 23 + (56 / 60) + (4 / 3600) # Hours of A Day

In [ ]:
Year_Days = 365.2425 # Days of 

In [ ]:
Year_Seconds = Year_Days * Day_Hours * 3600

In [ ]:
dD = 0.1 # Simulation Step In Day

In [ ]:
Stop_Year = 0.08 # The year that the simulation will stop.

In [ ]:
dY = dD / 365.2425 # Simulation  Step In Years

In [ ]:
Δt = dY * Year_Seconds # In Seconds

In [ ]:
Year = 0:dY:Stop_Year # In Years

In [ ]:
xM0 = -SEMD - (EMMD * cosd(M_i))

In [ ]:
yM0 = 0

In [ ]:
zM0 = EMMD * sind(M_i)

In [ ]:
VxM0 = 0

In [ ]:
VyM0 = -V_M0

In [ ]:
VzM0 = 0

In [ ]:
xE0 = -SEMD

In [ ]:
yE0 = 0

In [ ]:
zE0 = 0

In [ ]:
VxE0 = 0

In [ ]:
VyE0 = -V_E0

In [ ]:
VzE0 = 0

In [ ]:
x = zeros(12, length(Year))

In [ ]:
x[:, 1] = [xM0 yM0 zM0 xE0 yE0 zE0 VxM0 VyM0 VzM0 VxE0 VyE0 VzE0]'

In [ ]:
i = 1

In [ ]:
k1 = Δt .* f(x[:, i])

In [ ]:
k2 = Δt .* f(x[:, i] + k1 ./ 2)

In [ ]:
k3 = Δt .* f(x[:, i] + k2 ./ 2)